In [1]:
import numpy as np
import scipy.misc
import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
import scipy.misc
import imageio
import os
import itertools
from skimage.feature import local_binary_pattern
from skimage.feature import hog
from skimage import data, exposure
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score,f1_score
from sklearn.metrics import auc,precision_recall_curve
from skimage.transform import resize
from sklearn.neighbors import KNeighborsClassifier

In [2]:
def crop_face(img):
    img = img[:]
    faces = faceCascade.detectMultiScale(
            img,
            scaleFactor=1.1,
            minNeighbors=1,
            minSize=(30, 30))
    try:
        x,y,h,w = faces[0]
        face1 = zoom(img[y:y+h,x:x+w],[80./h,60./w])
        if face1.shape[1]!=60:
            face1 = resize(face1, (80, 60),anti_aliasing=True,preserve_range= True).astype('uint8')
        return face1
    except:
        img = resize(img, (80, 60),anti_aliasing=True,preserve_range= True).astype('uint8')
        return img

In [3]:
def create_lbp(data,x_grid = 5,y_grid = 5):
    hists = []
    for ind,img in enumerate(data):
        img = local_binary_pattern(crop_face(img),3,3)
        grid_size_x = img.shape[0]//x_grid
        grid_size_y = img.shape[1]//y_grid
        for x in range(x_grid):
            for y in range(y_grid):
                patch = img[x*grid_size_x:(x+1)*grid_size_x,y*grid_size_y:(y+1)*grid_size_y]
                hist,_ = np.histogram(patch.reshape(-1),bins=64)
                if x==0 and y==0:
                    patch_hist_list = hist
                else:
                    patch_hist_list = np.concatenate([patch_hist_list,hist],0)
        hists.append(patch_hist_list.reshape(1,-1))

    return np.concatenate(hists,0)

In [4]:
def create_hog(data):
    hog_features = [hog(crop_face(n), pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1)) for n in data]
    hog_features = np.stack(hog_features,0)
    hog_features = hog_features.reshape(len(data),-1)
    return hog_features

In [5]:
def create_label_pair(X,y):
    img_pair = []
    label_pair = []
    for pair in itertools.combinations_with_replacement(range(X.shape[0]),2):
        img_pair.append(X[pair,:])
        label_pair.append(y[pair,])

    img_pair = np.stack(img_pair)
    label_pair = np.stack(label_pair)
    label_pair = (label_pair[:,0] == label_pair[:,1])
    label_pair = label_pair*1
    img_pair = np.abs(img_pair[:,0,...] - img_pair[:,1,...])
    return (img_pair,label_pair)
    
    

In [6]:
cascPath = "haarcascade_frontalface_default.xml"

# Create the haar cascade
faceCascade = cv2.CascadeClassifier(cascPath)
full_face = []
full_labels = []
all_images = os.listdir('yalefaces')
for j in range(1,11):    # there are 15 different people in the dataset
    img_list = [filename for filename in all_images if filename.startswith('subject%.2d'%j)]
    for k,img in enumerate(img_list):
        I = imageio.imread(os.path.join('yalefaces',img))
        full_face.append(I)
    labels = [j-1]*11
    full_labels.extend(labels)
    

final_train=[]
final_labels = []
Data_main = os.listdir('att_faces')
for i in range(11,58):    # there are 15 different people in the dataset
    new_list='./att_faces/s'+str((i-10))
    img_list = os.listdir(new_list)
    for j in range(len(img_list)):
        I = imageio.imread(os.path.join(new_list,img_list[j]))
        final_train.append(I)
    labels = [i]*10
    final_labels.extend(labels)
full_face+=final_train
full_labels+=final_labels

In [7]:
hog_features = create_hog(full_face)
lbp_features = create_lbp(full_face)
# features = np.concatenate([hog_features,lbp_features],axis=1)
features= hog_features
# X_train, X_test, y_train, y_test = train_test_split(hog_features, np.array(full_labels), test_size=0.2, random_state=0)

/home/anand/anaconda3/lib/python3.6/site-packages/skimage/feature/_hog.py:150: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15. To supress this message specify explicitly the normalization method.
  skimage_deprecation)
/home/anand/anaconda3/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:583: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)
/home/anand/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [8]:
from sklearn.decomposition import PCA
pca = PCA(whiten=True)
train_feat = features
# test_feat = X_test
# print(train_feat.shape,test_feat.shape)
pca.fit(train_feat)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=True)

In [9]:
n_components = np.argmin(np.cumsum(pca.explained_variance_ratio_)<=0.99)
# print(n_components)
train_feat_pca = pca.transform(train_feat)
train_feat_pca = train_feat_pca[:,:n_components]
# test_feat_pca = pca.transform(test_feat)
# test_feat_pca = test_feat_pca[:,:n_components]
print(train_feat_pca.shape)

(580, 101)


In [10]:
img_pair, label_pair = create_label_pair(train_feat_pca,np.array(full_labels))
# test_img_pair, test_label_pair = create_label_pair(test_feat_pca,y_test)

In [11]:
g = np.random.permutation(img_pair.shape[0])
img_pair = img_pair[g,:]
label_pair = label_pair[g,]

# clf = SVC(probability=True)
clf=RandomForestClassifier(n_estimators = 300,n_jobs=-1,class_weight='balanced')
# clf = KNeighborsClassifier(n_neighbors=5)
# clf = GradientBoostingClassifier()
# clf = 
# clf = LogisticRegression()
clf.fit(img_pair,label_pair)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=300, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [12]:
# pred = clf.predict_proba(test_img_pair)
# print(clf.score(test_img_pair,test_label_pair))
# print(f1_score(clf.predict(test_img_pair),test_label_pair))

In [13]:
import h5py
import pandas as pd
import matplotlib.pyplot as plt
import time
%matplotlib inline
# from sklearn.metrics.pairwise import cosine_similarity
from skimage import io, exposure
from skimage.transform import resize
import cv2

df = pd.read_csv('image_pairs.csv')
uniq = set(df['pair1'].unique().tolist()+df['pair2'].unique().tolist())

i = 0
hf = h5py.File('test.h5', 'r')
data = list(map(hf.get,map(str,uniq)))

# hist_vector = create_vector(data)
hog_features = create_hog(data)
lbp_features = create_lbp(data)
dc = dict(zip(uniq,pca.transform(hog_features)[:,:n_components]))
# dc = dict(zip(uniq,pca.transform(hog_features)))

/home/anand/anaconda3/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:583: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)
/home/anand/anaconda3/lib/python3.6/site-packages/skimage/feature/_hog.py:150: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15. To supress this message specify explicitly the normalization method.
  skimage_deprecation)
/home/anand/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [14]:
testpred = []

for ind,row in df.iterrows():
    testpred.append(np.abs(dc[row['pair1']] - dc[row['pair2']]))
testpred = clf.predict_proba(testpred)
testpred = (testpred[:,1]>0.3).astype(int)
pd.DataFrame([range(df.shape[0]),testpred],index = ['Id','Expected']).T.to_csv('submisssion.csv',index=False)
